In [1]:
import get_data as driftlon_dataset
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, TimeDistributed
from datetime import datetime
import numpy as np

In [2]:
batch_size = 2000
match_depth = 50

(train_games, train_targets) = driftlon_dataset.get_data_batch(batch_size, match_depth)

batch_size = len(train_games)

train_games = np.array(train_games)
train_targets = np.array(train_targets)

encoded = []
for target in train_targets:
    target_vector = [0, 0]
    target_vector[target] = 1
    encoded.append(target_vector)
train_targets = np.array(encoded)

#train_targets = train_targets.reshape(batch_size,1)

In [20]:
activation = "None"

model = Sequential()

model.add(GRU(128, input_shape=(match_depth, 35), return_sequences=True))
model.add(Dropout(0.1))

model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.25))

model.add(GRU(128))
model.add(Dropout(0.25))

model.add(Dense(128))
model.add(Dense(128))
model.add(Dense(2))

#model.add(Dense(128, activation=activation))
#model.add(Dense(128, activation=activation))
#model.add(Dense(2, activation=activation))

#model.summary()

In [21]:
loss = tf.keras.losses.BinaryCrossentropy()
optimizer=tf.keras.optimizers.Adagrad()
metrics=['accuracy']

model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [22]:
suffix = 'gru-' + activation + '-' + datetime.now().strftime('%Y%m%d-%H%M%S')
#logdir='tb_logs/fit/' + suffix
logdir='tb_logs/activation_fns/' + suffix
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [23]:
hist = model.fit(train_games, train_targets, batch_size=50, epochs=10, callbacks=[tensorboard_callback])

if hist.history.get('accuracy')[-1] > 0.7:
    model.save('./neural_networks/saved_models/model-' + suffix)

Train on 695 samples
Epoch 1/10
695/695 [==============================] - 18s 26ms/sample - loss: 2.2138 - accuracy: 0.5000
Epoch 2/10
695/695 [==============================] - 6s 8ms/sample - loss: 1.6196 - accuracy: 0.5000
Epoch 3/10
695/695 [==============================] - 6s 8ms/sample - loss: 1.5504 - accuracy: 0.4993
Epoch 4/10
695/695 [==============================] - 6s 8ms/sample - loss: 1.4723 - accuracy: 0.5007
Epoch 5/10
695/695 [==============================] - 6s 8ms/sample - loss: 1.4353 - accuracy: 0.5000
Epoch 6/10
695/695 [==============================] - 6s 8ms/sample - loss: 1.4194 - accuracy: 0.5000
Epoch 7/10
695/695 [==============================] - 6s 8ms/sample - loss: 1.3900 - accuracy: 0.5000
Epoch 8/10
695/695 [==============================] - 6s 8ms/sample - loss: 1.3443 - accuracy: 0.4978
Epoch 9/10
695/695 [==============================] - 6s 8ms/sample - loss: 1.2958 - accuracy: 0.5014
Epoch 10/10
695/695 [==============================] - 6s 8

In [31]:
test_batch_size = 100

(test_games, test_targets) = driftlon_dataset.get_data_batch(test_batch_size)
test_games = np.array(test_games)
test_targets = np.array(test_targets)

test_encoded = []
for target in test_targets:
    target_vector = [0, 0]
    target_vector[target] = 1
    test_encoded.append(target_vector)
test_targets = np.array(test_encoded)

#train_targets = train_targets.reshape(batch_size,1)

In [32]:
model.evaluate(test_games, test_targets)

100/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.6860414290428162, 0.58]